In [ ]:
# Importamos bibliotecas.
using Plots, LaTeXStrings, DelimitedFiles

In [ ]:
# Calculamos la condición inicial de Carlos.

# Este es el sistema estático de Carlos.
function rhsStatic(w, r, a, α, ϕ, ψ)
    
    # Definimos las ecuaciones diferenciales de los estados estáticos.
    da = (1 - a^2)/(2*r) + (r/2)*(ψ^2 + (a*ϕ)^2*(1 + (w/α)^2))
    dα = (a^2 - 1)/r + da - r*(a*ϕ)^2
    dϕ = ψ
    dψ = -(2/r + dα - da)*ψ + a^2*(1 - (w/α)^2)*ϕ

    return [a*da, α*dα, dϕ, dψ]
end

# Definimos una función que haga rk4 para calcular el siguiente paso.
function rk4Static(dr, w, r, a, α, ϕ, ψ)

    k1  = rhsStatic(w, r, a, α, ϕ, ψ)
    k1a = k1[1]
    k1α = k1[2]
    k1ϕ = k1[3]
    k1ψ = k1[4]

    k2  = rhsStatic(w, r + dr/2, a + (k1a*dr/2), α + (k1α*dr/2), ϕ + (k1ϕ*dr/2), ψ + (k1ψ*dr/2))
    k2a = k2[1]
    k2α = k2[2]
    k2ϕ = k2[3]
    k2ψ = k2[4]

    k3  = rhsStatic(w, r + dr/2, a + (k2a*dr/2), α + (k2α*dr/2), ϕ + (k2ϕ*dr/2), ψ + (k2ψ*dr/2))
    k3a = k3[1]
    k3α = k3[2]
    k3ϕ = k3[3]
    k3ψ = k3[4]

    k4  = rhsStatic(w, r + dr, a + k3a*dr, α + k3α*dr, ϕ + k3ϕ*dr, ψ + k3ψ*dr)
    k4a = k4[1]
    k4α = k4[2]
    k4ϕ = k4[3]
    k4ψ = k4[4]

    rrk4 = r + dr
    ark4 = a + (k1a + 2*k2a + 2*k3a + k4a)*(dr/6)
    αrk4 = α + (k1α + 2*k2α + 2*k3α + k4α)*(dr/6)
    ϕrk4 = ϕ + (k1ϕ + 2*k2ϕ + 2*k3ϕ + k4ϕ)*(dr/6)
    ψrk4 = ψ + (k1ψ + 2*k2ψ + 2*k3ψ + k4ψ)*(dr/6)
    
    return [rrk4, ark4, αrk4, ϕrk4, ψrk4]
end

function solver(ϕi, w, Nr, dr)
    
    # Definimos las condiciones iniciales.
    r0 = dr
    a0 = 1
    α0 = 1
    ϕ0 = ϕi
    ψ0 = 0.0

    # Definimos los arreglos a usar.
    r = zeros(Nr)
    a = zeros(Nr)
    α = zeros(Nr)
    ϕ = zeros(Nr)
    ψ = zeros(Nr)

    # Llenamos las condiciones iniciales (y las simetrías).
    r[2] = r0
    a[2] = a0
    α[2] = α0
    ϕ[2] = ϕ0
    ψ[2] = ψ0

    r[1] = -r0
    a[1] = +a0
    α[1] = +α0
    ϕ[1] = +ϕ0
    ψ[1] = -ψ0

    # Comenzamos a solucionar en sí.
    for j in 2:Nr-1
        soluto = rk4Static(dr, w, r[j], a[j], α[j], ϕ[j], ψ[j])

        r[j+1] = soluto[1]
        a[j+1] = soluto[2]
        α[j+1] = soluto[3]
        ϕ[j+1] = soluto[4]
        ψ[j+1] = soluto[5]
    end

    return [r, a, α, ϕ, ψ]
end

function shooting(ϕi, w, Nr, dr)
    
    # Definimos la primer diferencial de 'w'.
    tolerancia = dr^4
    dw = 0.000001

    # Definimos un contador para ver cuántas veces necesitamos solucionar.
    counter = 0
    
    # Solucionamos por primera vez.
    ϕ = solver(ϕi, w, Nr, dr)[4]
    counter += 1

    while abs( ϕ[end] ) > tolerancia
        
        if ϕ[end] > tolerancia
            # Actualizamos la 'w'.
            w = w + dw

            # Volvemos a solucionar.
            ϕ = solver(ϕi, w, Nr, dr)[4]

        else
            # Aquí la 'w' ha sido muy grande, entonces debemos regresar.
            w = w - dw

            # Ahora hacemos más pequeña la varaición y volvemos a avanzar.
            dw = dw/2
            w = w + dw

            # Volvemos a solucionar.
            ϕ = solver(ϕi, w, Nr, dr)[4]
        end
        
        counter += 1
    end
        
    # Solucionamos una última vez (pero no la contamos).
    solucion = solver(ϕi, w, Nr, dr)
    r = solucion[1]
    a = solucion[2]
    α = solucion[3]
    ϕ = solucion[4]
    ψ = solucion[5]
    
    return [counter, w, r, a, α, ϕ, ψ]
end

function initial(ϕi, w, Nr, dr)

    # Aplicamos el shooting.
    soluto = shooting(ϕi, w, Nr, dr)
    counter = soluto[1]
    w = soluto[2]
    r = soluto[3]
    a = soluto[4]
    α = soluto[5]
    ϕ = soluto[6]
    ψ = soluto[7]
    println("Iteraciones del shooting: $counter \n")
    
    # Redimensionamos alpha.
    α00 = 1/(a[Nr]*α[Nr])
    α = α00 .* α

    # Calculamos la parte real e imaginaria de las funciones auxiliares.
    ϕ1 = copy(ϕ)
    ϕ2 = zeros(Nr)

    ψ1 = copy(ψ)
    ψ2 = zeros(Nr)

    π1 = zeros(Nr)
    π2 = -(w*α00) .* ϕ .* ( a ./ α )

    m = (r ./ 2) .* ( 1 .- (1 ./ a .^ 2) )
    σ = a .* α
    
    return [r, m, a, α, σ, ϕ1, ϕ2, ψ1, ψ2, π1, π2]
end

In [ ]:
# Definimos una función para los rhs.
function rhs(Nr, dr, r, a, α, ϕ1, ϕ2, ψ1, ψ2, π1, π2)

    # Definimos arreglos para guardar los lados derechos (los rhs: right-hand side).
    ϕ1_f = zeros(Nr)
    ϕ2_f = zeros(Nr)
    
    ψ1_f = zeros(Nr)
    ψ2_f = zeros(Nr)

    π1_f = zeros(Nr)
    π2_f = zeros(Nr)

    # Llenamos estos arreglos con (9) de Argelia-Siddhartha.
    for j in 1:Nr
        ϕ1_f[j] = (α[j]/a[j])*π1[j]
        ϕ2_f[j] = (α[j]/a[j])*π2[j]
    end

    for j in 2:Nr-1
        ψ1_f[j] = ( (α[j+1]/a[j+1])*π1[j+1] - (α[j-1]/a[j-1])*π1[j-1] )/(2*dr)
        ψ2_f[j] = ( (α[j+1]/a[j+1])*π2[j+1] - (α[j-1]/a[j-1])*π2[j-1] )/(2*dr)
    end

    for j in 2:Nr-1
        π1_f[j] = (1/r[j]^2)*( r[j+1]^2*(α[j+1]/a[j+1])*ψ1[j+1] -
                    r[j-1]^2*(α[j-1]/a[j-1])*ψ1[j-1] )/(2*dr) -
                    a[j]*α[j]*ϕ1[j]

        π2_f[j] = (1/r[j]^2)*( r[j+1]^2*(α[j+1]/a[j+1])*ψ2[j+1] -
                    r[j-1]^2*(α[j-1]/a[j-1])*ψ2[j-1] )/(2*dr) -
                    a[j]*α[j]*ϕ2[j]
    end

    return [ϕ1_f, ϕ2_f, ψ1_f, ψ2_f, π1_f, π2_f]
end

In [ ]:
# Definimos una función para solucionar la métrica.

# Las ecuaciones son (10) y (11) de Argelia-Siddhartha.
function metricrk2(Nr, dr, r, ϕ1, ϕ2, ψ1, ψ2, π1, π2)

    #------------------------------------------------
    # Solucionamos primero para a.
    a = zeros(Nr)
    a[2] = 1

    for j in 2:Nr-1
        
        rho = (r[j]/2)*( π1[j]^2 + π2[j]^2 + ψ1[j]^2 + ψ2[j]^2 +
                (a[j]^2)*(ϕ1[j]^2 + ϕ2[j]^2) )
        
        geo = ( a[j]^2 - 1 )/( 2*r[j] )

        aux_p = rho - geo

        k1  = a[j]*aux_p

        x   = r[j] + dr/2
        y   = a[j] + k1*dr/2

        #------------------------------------------------

        rho = (x/2)*( π1[j+1]^2 + π2[j+1]^2 + ψ1[j+1]^2 + ψ2[j+1]^2 +
                y^2*(ϕ1[j+1]^2 + ϕ2[j+1]^2) )
        
        geo = ( y^2 - 1 )/( 2*x )

        aux = rho - geo

        k2  = y*( aux + aux_p )/2

        #------------------------------------------------
        a[j+1] = a[j] + k2*dr

    end

    a[1] = a[2]

    #------------------------------------------------
    # Ahora solucionamos para alpha.
    # Necesitamos solucionar hacia atrás porque conocemos una condición
    # en la frontera exterior, no en el inicio.
    α = zeros(Nr)
    α[Nr] = 1/a[Nr]

    for j in Nr-1:-1:2

        rho = (r[j+1]/2)*( π1[j+1]^2 + π2[j+1]^2 + ψ1[j+1]^2 + ψ2[j+1]^2 -
                a[j+1]^2*(ϕ1[j+1]^2 + ϕ2[j+1]^2) )

        geo = ( a[j+1]^2 - 1 )/( 2*r[j+1] )

        aux_p = geo + rho

        k1 = α[j+1]*aux_p

        x  = r[j+1] - dr/2
        y  = α[j+1] - k1*dr/2

        #------------------------------------------------
        rho = (x/2)*( π1[j]^2 + π2[j]^2 + ψ1[j]^2 + ψ2[j]^2 -
                a[j]^2*(ϕ1[j]^2 + ϕ2[j]^2) )

        geo = ( a[j]^2 - 1 )/( 2*x )

        aux = geo + rho

        k2  = y*( aux + aux_p )/2

        #------------------------------------------------
        α[j] = α[j+1] - k2*dr
        
    end

    α[1] = α[2]
    
    return [a, α]
end

In [ ]:
# Definimos una función para el ICN en sí.
function icn(Nr, dr, dt, r, a, α, ϕ1, ϕ2, ψ1, ψ2, π1, π2)

    # Limpiamos adot.
    adot = zeros(Nr)

    # Guardamos la información previa.
    a_p  = copy(a)
    α_p  = copy(α)
    ϕ1_p = copy(ϕ1)
    ϕ2_p = copy(ϕ2)
    ψ1_p = copy(ψ1)
    ψ2_p = copy(ψ2)
    π1_p = copy(π1)
    π2_p = copy(π2)
    
    #------------------------------------------------
    # Iniciamos el ciclo interno del ICN.
    for k in 1:3

        # Calculamos el tamaño del paso temporal en función de la iteración.
        if k < 3
            dtw = dt/2
        else
            dtw = dt
        end

        # Calculamos los lados derechos.
        fuentes = rhs(Nr, dr, r, a, α, ϕ1, ϕ2, ψ1, ψ2, π1, π2)
        ϕ1_f = fuentes[1]
        ϕ2_f = fuentes[2]
        ψ1_f = fuentes[3]
        ψ2_f = fuentes[4]
        π1_f = fuentes[5]
        π2_f = fuentes[6]

        # Actualizamos los campos.
        for j in 2:Nr-1
            ϕ1[j] = ϕ1_p[j] + dtw*ϕ1_f[j]
            ϕ2[j] = ϕ2_p[j] + dtw*ϕ2_f[j]
            ψ1[j] = ψ1_p[j] + dtw*ψ1_f[j]
            ψ2[j] = ψ2_p[j] + dtw*ψ2_f[j]
            π1[j] = π1_p[j] + dtw*π1_f[j]
            π2[j] = π2_p[j] + dtw*π2_f[j]
        end
        
        # Imponemos las condiciones en la frontera interior.
        ϕ1[1] = ϕ1_p[1] + dtw*ϕ1_f[1]
        ϕ2[1] = ϕ2_p[1] + dtw*ϕ2_f[1]
        ψ1[1] = -ψ1[2]
        ψ2[1] = -ψ2[2]
        π1[1] = +π1[2]
        π2[1] = +π2[2]

        # Imponemos las condiciones en la frontera exterior.
        div = 1 + dtw/dr + dtw/r[Nr]
        ϕ1[Nr] = ϕ1_p[Nr] + dtw*ϕ1_f[Nr]
        ϕ2[Nr] = ϕ2_p[Nr] + dtw*ϕ2_f[Nr]
        π1[Nr] = (π1[Nr-1]*(dtw/dr - 1) + π1_p[Nr]*(1 - dtw/dr - dtw/r[Nr]) + π1_p[Nr-1]*(dtw/dr + 1))/div
        π2[Nr] = (π2[Nr-1]*(dtw/dr - 1) + π2_p[Nr]*(1 - dtw/dr - dtw/r[Nr]) + π2_p[Nr-1]*(dtw/dr + 1))/div
        ψ1[Nr] = -π1[Nr] - ϕ1[Nr]/r[Nr]
        ψ2[Nr] = -π2[Nr] - ϕ2[Nr]/r[Nr]

        # Calculamos la métrica.
        metricas = metricrk2(Nr, dr, r, ϕ1, ϕ2, ψ1, ψ2, π1, π2)
        a = metricas[1]
        α = metricas[2]        

    #------------------------------------------------
    # Terminamos el ciclo interno.
    end
    
    # Calculamos la masa y la métrica.
    m = (r ./ 2) .* ( 1 .- (1 ./ a .^2) )
    σ = a .* α

    # Calculamos adot.
    Srr_p = r .* α_p .* (ψ1_p .* π1_p .+ ψ2_p .* π2_p)
    Srr = r .* α .* (ψ1 .* π1 .+ ψ2 .* π2)

    adot = ((a .- a_p) ./ dt) .- (1/2) .* (Srr_p .+ Srr)
    
    return [m, a, α, σ, ϕ1, ϕ2, ψ1, ψ2, π1, π2, adot]
end

In [ ]:
# Definimos la función main.
function main(ϕi, w, rf, dr, tf, courant)
    
    # Definimos la longitud de los arreglos.
    Nr = round(Int, rf/dr)
    Nt = round(Int, tf*(Nr/(courant*rf)))
    dt = courant*dr

    # Calculamos la condición de guardado temporal y espacial.
    savedataT = round(Int, Nt/100)
    savedataR = round(Int, Nr/100)

    # Definimos arreglos que guardarán la información.
    adot_dat = []
    phi1_dat = []

    # Iniciamos el tiempo en cero.
    t = 0
    
    # Calculamos las condiciones iniciales.
    iniciales = initial(ϕi, w, Nr, dr)
    r  = iniciales[1]
    m  = iniciales[2]
    a  = iniciales[3]
    α  = iniciales[4]
    σ  = iniciales[5]
    ϕ1 = iniciales[6]
    ϕ2 = iniciales[7]
    ψ1 = iniciales[8]
    ψ2 = iniciales[9]
    π1 = iniciales[10]
    π2 = iniciales[11]
    adot = zeros(Nr)

    # Guardamos las condiciones iniciales.
    for j in 2:Nr
        push!(adot_dat, [t, r[j], adot[j]])
    end
    push!(phi1_dat, [t, ϕ1[2]])

    # Graficamos las condiciones iniciales.
    grafica = plot(layout = (1, 3), size = (1500, 500))

    plot!(grafica[1], r, adot, label = "adot(r)", linecolor=:teal)
    
    plot!(grafica[2], r, a, label = "a(r)", linecolor=:magenta)
    plot!(grafica[2], r, σ, label = "σ(r)", linecolor=:sienna)
    plot!(grafica[2], r, α, label = "α(r)", linecolor=:teal)

    plot!(grafica[3], r, ϕ1, label = "ϕ1(r)", linecolor=:magenta)
    plot!(grafica[3], r, ϕ2, label = "ϕ2(r)", linecolor=:teal)

    # Iniciamos el ciclo principal de evolución.
    for n in 2:Nt

        # Avanzamos en el tiempo.
        t = t + dt

        # Hacemos el ICN.
        crank = icn(Nr, dr, dt, r, a, α, ϕ1, ϕ2, ψ1, ψ2, π1, π2)
        m  = crank[1]
        a  = crank[2]
        α  = crank[3]
        σ  = crank[4]
        ϕ1 = crank[5]
        ϕ2 = crank[6]
        ψ1 = crank[7]
        ψ2 = crank[8]
        π1 = crank[9]
        π2 = crank[10]
        adot = crank[11]

        # Graficamos y gurdamos.
        if n % savedataT == 0
            for j in 2:Nr
                push!(adot_dat, [t, r[j], adot[j]])
            end
            push!(phi1_dat, [t, ϕ1[2]])
            
            plot!(grafica[1], r, adot, label="", linecolor=:teal)
            
            plot!(grafica[2], r, a, label="", linecolor=:magenta)
            plot!(grafica[2], r, σ, label="", linecolor=:sienna)
            plot!(grafica[2], r, α, label="", linecolor=:teal)

            plot!(grafica[3], r, ϕ1, label="", linecolor=:magenta)
            plot!(grafica[3], r, ϕ2, label="", linecolor=:teal)
        end

    end

    # Mostramos la gráfica.
    display(grafica)

    # Escribimos los archivos en sí.
    writedlm("adot.dat", adot_dat)
    writedlm("phi1.dat", phi1_dat)
    
    return
end

In [ ]:
# Ejecutamos la función principal.
@time begin
    ϕi = 0.110             # Parámetro inicial del campo.
    w  = 1.0809049555648   # Frecuencia de oscilación.
    rf = 45                # Radio de integración final.
    dr = 0.01              # Tamaño del paso.
    tf = 10                # Tiempo final de integración.
    courant = 0.5          # parámetro de courant.
        
    main(ϕi, w, rf, dr, tf, courant)
end